### Extract the data from the CSV

In [4]:
import numpy as np
from sklearn import preprocessing

raw_csv_data = np.loadtxt('Audiobooks_data.csv', delimiter =',')

unscalled_inputs_all = raw_csv_data[:,1:-1] ## Seprate the ID number, and target values
targets_all = raw_csv_data[:,-1]    ## Seunsc only the target values

### Balance the dataset

In [11]:
## need to calculate the 1's from targets
num_one_targets = int(np.sum(targets_all))

## need to keep same count of 0's(delete the others) 
zero_target_counter = 0
## create the varibale that records the variables need to remove
indices_to_remove = []  ## we need to have list of tuple, so we created the empty brackets

## need to iterate over the dataset and balance it
for i in range(targets_all.shape[0]):
    if targets_all[i] == 0:
        zero_target_counter += 1
        if zero_target_counter > num_one_targets:
            indices_to_remove.append(i)  ## number target 0 count is equals to 1 count, will note the indices to indices_to_rm

## we remove the indices from the original list of inputs & targets
unscalled_inputs_equal_priors = np.delete(unscalled_inputs_all, indices_to_remove, axis=0)
targets_equal_priors = np.delete(targets_all, indices_to_remove, axis=0)

### Standardize the inputs

In [14]:
### with the help of preprocessing tool we can standardize the data 
scaled_inputs = preprocessing.scale(unscalled_inputs_equal_priors)

### Shuffle the data

In [15]:
## There might be a possiblity that our data could be in order, since we are batching we need to shuffle the data
## if the data is in order it will confuse the SGD, when we average the loss across the batches
 
shuffled_indices = np.arange(scaled_inputs.shape[0]) ## scaled input values are stored in a variable
np.random.shuffle(shuffled_indices)       ### then the data is shuffled using the random.shuffle method by using indices
 
shuffled_inputs = scaled_inputs[shuffled_indices]   ## shuffle the inputs using indices
shuffled_targets = targets_equal_priors[shuffled_indices] ## shuffle the targets

### Split the data into Training, Validation and Test

In [20]:
samples_count = shuffled_inputs.shape[0]

## we must determine the size of the datasetf
## here we split the data as 80:10:10 for train:validation:test

train_samples_count = int(0.8*samples_count)
validation_samples_count = int(0.1*samples_count)
test_samples_count = samples_count - train_samples_count -validation_samples_count

## we need to extract them from the big dataset

## train samples can be extracted from the selected train_samples_count
train_inputs = shuffled_inputs[:train_samples_count]
train_targets = shuffled_targets[:train_samples_count]

## validation samples are collected using the below method
validation_inputs = shuffled_inputs[train_samples_count:train_samples_count+validation_samples_count]
validation_targets = shuffled_targets[train_samples_count:train_samples_count+validation_samples_count]

## test data is the remaining data
test_inputs = shuffled_inputs[train_samples_count+validation_samples_count:]
test_targets = shuffled_targets[train_samples_count+validation_samples_count:]

### print and check the data which is left
print("Train_data: ",np.sum(train_targets), train_samples_count, np.sum(train_targets) / train_samples_count)
print("validation_data: ",np.sum(validation_targets), validation_samples_count, np.sum(validation_targets)/validation_samples_count)
print("Test_data: ",np.sum(test_targets), test_samples_count, np.sum(test_targets)/test_samples_count)

Train_data:  1796.0 3579 0.5018161497625034
validation_data:  232.0 447 0.5190156599552572
Test_data:  209.0 448 0.46651785714285715


### Save three dataset in .npz format

In [21]:
np.savez('Audiobooks_data_train', inputs=train_inputs, targets=train_targets)
np.savez('Audiobooks_data_validation', inputs=validation_inputs, targets=validation_targets)
np.savez('Audiobooks_data_test', inputs=test_inputs, targets=test_targets)